In [3]:
import rdflib
from nanopub import Publication, NanopubClient

## Using the nanopublication namespace within assertions

Upon publication, any blank nodes in the rdf graph are replaced with the nanopub's URI, with the blank node name as a
fragment. For example, if the blank node is called 'step', that would result in a URI composed of the nanopub's (base)
URI, followed by #step. We can thus use blank nodes to refer to new concepts, making use of the namespace of the 
to-be-published URI.

In [4]:
my_assertion = rdflib.Graph()

# We want to introduce a new concept in our publication: Tim Berners Lee
tim = rdflib.BNode('timbernerslee')

# We assert that he is a person
my_assertion.add((tim, rdflib.RDF.type, rdflib.FOAF.Person) )

# And create a publication object for this assertion
publication = Publication.from_assertion(assertion_rdf=my_assertion)

/Users/svenvanderburg/projects/fair-workflows/nanopub/nanopub/profile.py:51: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)


In [5]:
# The assertion now uses a dummy namespace (http://purl.org/nanopub/temp/mynanopub#) that will later be replaced with the nanopub uri
print(publication.assertion.serialize(format='trig').decode())

@prefix : <http://purl.org/nanopub/temp/mynanopub#> .

:assertion {
    :timbernerslee a <http://xmlns.com/foaf/0.1/Person> .
}




In [6]:
# Let's publish this to the test server
client = NanopubClient(use_test_server=True)
pubinfo = client.publish(publication)

Published to http://purl.org/np/RAFGh8DInBCSiRSD4ih315Z3zNcM3DZ-NN6HpZYDxq7Mk


In [7]:
# Let's fetch the freshly published nanopub:
published = client.fetch(pubinfo['nanopub_uri'])

# As you see the dummy namespace has been replaced with the nanopub URI namespace!
print(published.assertion.serialize(format='trig').decode())

@prefix sub: <http://purl.org/np/RAFGh8DInBCSiRSD4ih315Z3zNcM3DZ-NN6HpZYDxq7Mk#> .

sub:assertion {
    sub:timbernerslee a <http://xmlns.com/foaf/0.1/Person> .
}




## Introducing a concept
You can optionally specify that the ```Publication``` introduces a particular concept using blank nodes. 
The pubinfo graph will note that this nanopub npx:introduces the concept. The concept should be a blank node 
(rdflib.term.BNode), and is converted to a URI derived from the nanopub's URI with a fragment (#) made from the blank
node's name.

In [8]:
# We can create a publication introducing this new concept
publication = Publication.from_assertion(assertion_rdf=my_assertion,
                                         introduces_concept=tim)

In [9]:
# Let's publish this to the test server
client = NanopubClient(use_test_server=True)
pubinfo = client.publish(publication)

Published to http://purl.org/np/RAdpeRop9Ow5ArTRivUWkw7ArCI--J5K1vN3J9fw9gnuk
Published concept to http://purl.org/np/RAdpeRop9Ow5ArTRivUWkw7ArCI--J5K1vN3J9fw9gnuk#timbernerslee


In [10]:
# pubinfo now contains the concept uri
pubinfo['concept_uri']

'http://purl.org/np/RAdpeRop9Ow5ArTRivUWkw7ArCI--J5K1vN3J9fw9gnuk#timbernerslee'

In [11]:
# Let's fetch the freshly published nanopub:
published = client.fetch(pubinfo['nanopub_uri'])

In [12]:
# The publication info subgraph notes that this nanopub introduces Tim Berners Lee
print(published.pubinfo.serialize(format='trig').decode())

@prefix npx: <http://purl.org/nanopub/x/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix sub: <http://purl.org/np/RAdpeRop9Ow5ArTRivUWkw7ArCI--J5K1vN3J9fw9gnuk#> .
@prefix this: <http://purl.org/np/RAdpeRop9Ow5ArTRivUWkw7ArCI--J5K1vN3J9fw9gnuk> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:pubInfo {
    sub:sig npx:hasAlgorithm "RSA" ;
        npx:hasPublicKey "MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQCmso7vmRO/Cp4Pt0RkJJkV5qfc1WFYU/jMtkdxxb5+lfIVXNV97XQnM1Tj4fkb/W6jkP6fHl8mj8Q7hl7VgUnQ6I+B7cMGpxW9Z8Br+JNx8DPMMt08VCH5+JMENPRKl91r7rF/YPWCAgL9eqXSixCNMNAj5RBmMTQoPuRkpgmt1wIDAQAB" ;
        npx:hasSignature "ECl9+3rpTErXr72pZWXlNVVEVWu0p1L1y1r46wzCs9Gj4A/Q+joB6gcf51S/HVnNgq0mHSPWJspcqvi5bp8i3ogMI5nz8SGWbgSoDsFVW9XN7xoy7mQ2LUfCTUZYNRic2nH9Bo3k9RnooSP79dFpINC7g0RRnNcl/D+SVUPYqBQ=" ;
        npx:hasSignatureTarget this: .

    this: npx:introduces sub:timbernerslee ;
        prov:generatedAtTime "2020-11-30T12:42:42.151969"^^xsd:dateTime ;
        prov:wasAttributedTo <https